The purpose of this project is to answer the following question: what is the combination of foods with the fewest calories such that all recommended daily values (RDV) for every essential micronutrient is satisfied? 

My inspiration for this project is a desire to create a healthy diet plan that's as easy-to-follow as possible. As I understand it, a diet can be considered "healthy" if it satisfies two conditions: first, it must include an adequate amount of necessary nutrients (vitamins and minerals, as well as macronutrients); and second, it must limit the amount of certain groups of foods (like processed foods, refined carbohydrates, etc.). I, along with much of the developed world, likely don't get enough of the vitamins and minerals I need from the food I eat, and so if I can find combinations of foods that give me adequate nutrition with low-calories, then I can satisfy the rest of my macronutrient profile relatively easily.

This problem is a Linear Programming problem. Linear Programming (LP) problems are problems in which there is an objective function you are trying to minimize/maximize, with a list of constraints that must be satisfied. In this project, the objective is that we are trying to minimize the number of calories in a list of foods, with the constraints being that that list must have at least the RDV for every micronutrient. Inside of pulp_helpers.py, there is a dictionary containing each relevant micronutrient and its RDV.

For this project I am using food nutrition data from the USDA (source at the end of this file). The databases that I will be using are the Foundation Foods, SR Legacy, FNDDS, and All Data Types. The details of what these databases are can be found on the USDA's website, but for the sake of this project all you need to know is that Foundation Foods is the only database with data about all 27 vitamins and minerals that we are interested in. Because of this, most of the analysis will be done by combining the data from the Foundation Foods database with data from the All Data Types Database.

In this project I use the PuLP library, which is used to answer  linear programming problems. I also use pandas and numpy for downloading and manipulating data. I created two custom modules to clean up the code in this file; one module contains the functions that extract and clean the data, and one module contains the functions that creates and solves the LP problem. These modules can be found in this repository.

In [1]:
import pandas as pd
import os
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpConstraint, LpStatus
import math
import numpy as np
import pulp_helpers as ph
import get_data_helpers as gdh
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

First, I'll show you what the Foundation Foods table looks like. The get_data_helpers.py file contains functions that extract the data from the USDA .csv files in this repository, then it manipulates the data into a form that's readable by the PuLP functions. If you want to see exactly how this is done, you can see in the get_data_helpers.py file.

In [2]:
df_ff = gdh.get_foundation_foods_table()
df_ff.head()

,fdc_id,description,"Ergosta-5,7-dienol (MG)","Ergosta-7,22-dienol (MG)",10-Formyl folic acid (10HCOFA) (UG),25-hydroxycholecalciferol (UG),5-Formyltetrahydrofolic acid (5-HCOH4 (UG),5-methyl tetrahydrofolate (5-MTHF) (UG),Alanine (G),Arginine (G),Ash (G),Aspartic acid (G),Beta-glucan (G),Beta-sitostanol (MG),Beta-sitosterol (MG),Betaine (MG),Biotin (UG),"Boron, B (UG)",Brassicasterol (MG),"Calcium, Ca (MG)",Campestanol (MG),Campesterol (MG),"Carbohydrate, by difference (G)","Carbohydrate, by summation (G)","Carotene, alpha (UG)","Carotene, beta (UG)","Carotene, gamma (UG)",Cholesterol (MG),"Choline, free (MG)","Choline, from glycerophosphocholine (MG)","Choline, from phosphocholine (MG)","Choline, from phosphotidyl choline (MG)","Choline, from sphingomyelin (MG)","Choline, total (MG)",Citric acid (MG),"Cobalt, Co (UG)","Copper, Cu (MG)","Cryptoxanthin, alpha (UG)","Cryptoxanthin, beta (UG)",Cysteine (G),Cystine (G),Daidzein (MG),Daidzin (MG),Delta-5-avenasterol (MG),Delta-7-Stigmastenol (MG),Energy (Atwater General Factors) (KCAL),Energy (Atwater Specific Factors) (KCAL),Energy (KCAL),Energy (kJ),Ergosta-7-enol (MG),Ergosterol (MG),Ergothioneine (MG),"Fatty acids, total monounsaturated (G)","Fatty acids, total polyunsaturated (G)","Fatty acids, total saturated (G)","Fatty acids, total trans (G)","Fatty acids, total trans-dienoic (G)","Fatty acids, total trans-monoenoic (G)","Fatty acids, total trans-polyenoic (G)","Fiber, insoluble (G)","Fiber, soluble (G)","Fiber, total dietary (G)","Folate, total (UG)",Fructose (G),Galactose (G),Genistein (MG),Genistin (MG),Glucose (G),Glutamic acid (G),Glycine (G),Glycitin (MG),High Molecular Weight Dietary Fiber (HMWDF) (G),Histidine (G),Hydroxyproline (G),"Iodine, I (UG)","Iron, Fe (MG)",Isoleucine (G),Lactose (G),Leucine (G),Low Molecular Weight Dietary Fiber (LMWDF) (G),Lutein (UG),Lutein + zeaxanthin (UG),Lycopene (UG),Lysine (G),MUFA 12:1 (G),MUFA 14:1 c (G),MUFA 15:1 (G),MUFA 16:1 c (G),MUFA 17:1 (G),MUFA 17:1 c (G),MUFA 18:1 (G),MUFA 18:1 c (G),MUFA 20:1 (G),MUFA 20:1 c (G),MUFA 22:1 (G),MUFA 22:1 c (G),MUFA 22:1 n-11 (G),MUFA 22:1 n-9 (G),MUFA 24:1 c (G),"Magnesium, Mg (MG)",Malic acid (MG),Maltose (G),"Manganese, Mn (MG)",Methionine (G),"Molybdenum, Mo (UG)",Niacin (MG),"Nickel, Ni (UG)",Nitrogen (G),Oxalic acid (MG),PUFA 18:2 (G),PUFA 18:2 CLAs (G),PUFA 18:2 c (G),"PUFA 18:2 n-6 c,c (G)",PUFA 18:3 (G),PUFA 18:3 c (G),"PUFA 18:3 n-3 c,c,c (ALA) (G)","PUFA 18:3 n-6 c,c,c (G)",PUFA 18:3i (G),PUFA 18:4 (G),PUFA 20:2 c (G),"PUFA 20:2 n-6 c,c (G)",PUFA 20:3 (G),PUFA 20:3 c (G),PUFA 20:3 n-3 (G),PUFA 20:3 n-9 (G),PUFA 20:4 (G),PUFA 20:4 n-6 (G),PUFA 20:4c (G),PUFA 20:5 n-3 (EPA) (G),PUFA 20:5c (G),PUFA 22:2 (G),PUFA 22:3 (G),PUFA 22:4 (G),PUFA 22:5 c (G),PUFA 22:5 n-3 (DPA) (G),PUFA 22:6 c (G),PUFA 22:6 n-3 (DHA) (G),Pantothenic acid (MG),Phenylalanine (G),"Phosphorus, P (MG)",Phytoene (UG),Phytofluene (UG),"Phytosterols, other (MG)","Potassium, K (MG)",Proline (G),Protein (G),Pyruvic acid (MG),Quinic acid (MG),Raffinose (G),Retinol (UG),Riboflavin (MG),SFA 10:0 (G),SFA 11:0 (G),SFA 12:0 (G),SFA 14:0 (G),SFA 15:0 (G),SFA 16:0 (G),SFA 17:0 (G),SFA 18:0 (G),SFA 20:0 (G),SFA 21:0 (G),SFA 22:0 (G),SFA 23:0 (G),SFA 24:0 (G),SFA 4:0 (G),SFA 5:0 (G),SFA 6:0 (G),SFA 7:0 (G),SFA 8:0 (G),SFA 9:0 (G),"Selenium, Se (UG)",Serine (G),"Sodium, Na (MG)",Specific Gravity (SP_GR),Stachyose (G),Starch (G),Stigmastadiene (MG),Stigmasterol (MG),Sucrose (G),"Sugars, Total (G)","Sugars, total including NLEA (G)","Sulfur, S (MG)",TFA 14:1 t (G),TFA 16:1 t (G),TFA 18:1 t (G),TFA 18:2 t (G),TFA 18:2 t not further defined (G),TFA 18:3 t (G),TFA 20:1 t (G),TFA 22:1 t (G),Thiamin (MG),Threonine (G),"Tocopherol, beta (MG)","Tocopherol, delta (MG)","Tocopherol, gamma (MG)","Tocotrienol, alpha (MG)","Tocotrienol, beta (MG)","Tocotrienol, delta (MG)","Tocotrienol, gamma (MG)",Total dietary fiber (AOAC 2011.25) (G),Total fat (NLEA) (G),Total lipid (fat) (G),Tryptophan (G),Tyrosine (G),Valine (G),Verbascose (G),"Vitamin

As you can see above, each row contains information about one specific food, and the columns contain data about how much of each nutrient is in that row's food. The majority of the nutrient columns aren't used by the PuLP functions, so we can ignore all of them except the relevant micronutrients as well as the calories column.

NOTE: many of the foods don't have information about some specific nutrients, and unless it has missing calorie info, for these instances I am treating that as if there is none of that nutrient in the food. This is likely untrue, but for the sake of this project it doesn't matter. If there is missing calorie information for a food, then we calculate it explicitly.

Now that we have the Foundation Foods table, we can input that table into our PuLP helper function to solve the problem for us:

In [3]:
ff_ans = ph.default_problem(df_ff, [])
print(ff_ans)

-Beef, round, top round roast, boneless, separable lean only, trimmed to 0" fat, select, raw: 52.42 g
-Lettuce, cos or romaine, raw: 525.88 g
-Salt, table, iodized: 1.51 g
-Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim): 579.73 g
-Beans, Dry, Red (0% moisture): 12.59 g
-Broccoli, raw: 294.13 g
-Eggs, Grade A, Large, egg whole: 122.65 g
-Almond milk, unsweetened, plain, shelf stable: 412.65 g
-Spinach, baby: 122.85 g
-Mushroom, maitake: 111.28 g
Total Calories: 777.7
Total Mass: 2236 g (4.9 lbs)


As we can see, using only the Foundation Foods table, the optimal list of foods is 52.4 g of lean beef, 525.9 g of romaine lettuce, etc., and this list gives you your necessary macronutrient profile at 777.7 calories. This list and the rest of the lists are repeated in the results.txt file.

You may notice that this list of foods weighs 4.9 lbs. We may want to come up with another list that gives the same micronutrient profile, but weighing less. The following request adds the constraing that the mass must be less than 1.5 kg

In [5]:
print(ph.default_problem(df_ff, [('mass (G)', -1, 1500)]))

-Kale, raw: 216.74 g
-Seeds, sunflower seed kernels, dry roasted, with salt added: 40.87 g
-Beef, round, top round roast, boneless, separable lean only, trimmed to 0" fat, select, raw: 36.44 g
-Salt, table, iodized: 1.49 g
-Beans, Dry, Brown (0% moisture): 13.67 g
-Broccoli, raw: 609.83 g
-Eggs, Grade A, Large, egg whole: 150.86 g
-Mushroom, lion's mane: 41.9 g
-Almond milk, unsweetened, plain, shelf stable: 137.45 g
-Mushroom, maitake: 52.93 g
-Soy milk, unsweetened, plain, refrigerated: 197.81 g
Total Calories: 952.7
Total Mass: 1500 g (3.3 lbs)


This list of foods weighs 3.3 lbs, and is 952.7 calories. Before we move on, I want to remove processed foods from this list. If I want my diet to be healthy, I want to remove as many processed foods as I can. In order to get an answer with no ultraprocessed foods (NOVA group 4 foods), I need to remove Almond milk and Soy milk.

In [6]:
remove_foods = [
    'Almond milk, unsweetened, plain, shelf stable',
    'Soy milk, unsweetened, plain, refrigerated'
]

temp_df = df_ff[~df_ff['description'].isin(remove_foods)].reset_index(drop=True)

string = ph.default_problem(temp_df, [])
print(string)

-Tomatoes, grape, raw: 31.23 g
-Kale, frozen, cooked, boiled, drained, without salt: 906.49 g
-Beef, round, top round roast, boneless, separable lean only, trimmed to 0" fat, select, raw: 38.11 g
-Salt, table, iodized: 2.17 g
-Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim): 855.15 g
-Beans, Dry, Medium Red (0% moisture): 14.46 g
-Eggs, Grade A, Large, egg whole: 109.52 g
-Oil, sunflower: 0.03 g
-Mushroom, maitake: 181.21 g
Total Calories: 929.9
Total Mass: 2138 g (4.7 lbs)


Next we will look at the All Data Types database. NOTE: in the get_data_helpers.py function get_all_databases_table(), it only includes foods that are in a select group of categories (namely fruits, vegetables, meats, and other clearly unprocessed/low-processed foods).

In [7]:
df_ad = gdh.get_all_databases_table()


In [ ]:
ad_ans = ph.default_problem(df_ad, [])
print(ad_ans)

As you can see, if we only use the "all databases" table, then we don't have adequate information about 6 of the nutrients. So we'll concatenate the foundation foods database onto this one:

In [ ]:
mask = df_ad['description'].isin(df_ff['description'])
df = pd.concat([df_ff, df_ad[~mask]], ignore_index=True, sort=False).fillna(0)

In [ ]:
string = ph.default_problem(df, [])
print(string)

In [ ]:
string = ph.default_problem(df, [('mass (G)', -1, 1500), ('Sodium, Na (MG)', -1, 5000)])
print(string)

In [ ]:
df_thing = nutrition_of_list(df, string)

#display(temp_df[temp_df['description'].isin(df_thing['description'])])
print(gdh.find_PDV(df_thing))


In [ ]:
remove_foods = [
    'Almond milk, unsweetened, plain, shelf stable',
    'Soy milk, unsweetened, plain, refrigerated',
    'Kidney',
    'Scallops, steamed or boiled',
    'Scallops, fried',
    'Rice milk',
    'Coconut milk'
]

tdf = df[~df['description'].isin(remove_foods)].reset_index(drop=True)

ans = ph.default_problem(tdf, [])
print(ans)

In [ ]:
df_thing = gdh.nutrition_of_list(df, ans)
#display(df_thing)
#display(tdf[tdf['description'].isin(df_thing['description'])])
print(gdh.find_PDV(df_thing))


In [ ]:
def filter_dataframe(df, tuples):
    # Extract the food names from the tuples
    foods = [t[0] for t in tuples]
    
    
    # Filter the dataframe based on the "description" column
    new_df = df[df['description'].isin(foods)]
    
    
    return new_df

def nutrition_of_list(df, input_string):

    tuples = gdh.extract_tuples(input_string)
    
    new_df = filter_dataframe(df, tuples)  # Create a copy of the original dataframe
    
    
    keep_cols = ['Energy (KCAL)', 'mass (G)', 'description']
    
    for food, factor in tuples:
        # Multiply the numeric columns by the factor for the matching food
        mask = new_df['description'] == food
        numeric_cols = new_df.select_dtypes(include='number').columns
        new_df.loc[mask, numeric_cols] *= (factor / 100)
        
        
    columns_list = new_df.columns.tolist()
    
    for col in columns_list:
        if (col not in keep_cols) and (col not in gdh.nutrients_dict.keys()):
            gdh.drop_cols(new_df, col)
        
    
    return new_df

In [ ]:
string = ph.default_problem(df1, [('mass (G)', -1, 1500)])

lst = gdh.extract_tuples(string)
for (food, number) in lst:
    print(f'{food}: {number} g')

In [ ]:
df_thing = gdh.nutrition_of_list(df1, string)
display(df_thing)

In [ ]:
print(gdh.find_PDV(df_thing))

In [ ]:
display(df_thing)

In [ ]:
df = gdh.get_foundation_foods_table()

string = ph.default_problem(df, [])

df2 = gdh.nutrition_of_list(df1, string)

string2 = gdh.find_PDV(df2)

print(string)

In [ ]:
print(gdh.find_PDV(df2))

In [ ]:
df1 = gdh.get_all_databases_table()
df = gdh.get_foundation_foods_table()
df = pd.concat([df, df1], ignore_index=True, sort=False).fillna(0)




In [ ]:
ans = ph.default_problem(df, [('mass (G)', -1, 1500)])

df_thing = gdh.nutrition_of_list(df, ans)

#display(df_thing)
print(gdh.find_PDV(df_thing))

In [ ]:
print(ans)

Using this source for what nutrients are necessary:
https://www.hsph.harvard.edu/nutritionsource/vitamins/

and this source for all data:
https://fdc.nal.usda.gov/download-datasets.html

There are very few foods in the USDA database that have a value for chromium listed; I've taken it out of my consideration, since the NIH says that "Chromium deficiency has not been reported in healthy populations".
https://ods.od.nih.gov/factsheets/Chromium-HealthProfessional/#:~:text=Many%20whole%20grains%2C%20fruits%2C%20and,poultry%2C%20and%20eggs%20contain%20chromium.